In [2]:
import cv2
from torchvision import transforms
import easyocr
import csv
from datetime import datetime
from ultralytics import YOLO

model = YOLO("./model.pt")

# Video settings
wCam, hCam = 1280, 750
cap = cv2.VideoCapture('./test_img_vid/test_vid_4.mp4')
cap.set(3, wCam)
cap.set(4, hCam)

# Class labels
class_labels = ['Plate','Helmet','Head','Rider']  # Update with your class labels

helmet = False
head = False
plate = False

reader = easyocr.Reader(['en'])

plate_text = ''

X1,Y1,X2,Y2 = 0,0,0,0


def log_data(plate, helmet, head, plate_text):
    if plate and helmet:
        new_rows = [[datetime.now(), plate_text, 'HELMET']]
        with open("./records.csv", mode='a', newline='') as file:
            writer = csv.writer(file)
            for row in new_rows:
                writer.writerow(row)

    elif plate and head:
        new_rows = [[datetime.now(), plate_text, 'WITHOUT HELMET']]
        with open("./records.csv", mode='a', newline='') as file:
            writer = csv.writer(file)
            for row in new_rows:
                writer.writerow(row)

while True:
    success, original_img = cap.read()
    if not success:
        break

    transform = transforms.ToTensor()
    rider = model(original_img)
    boxes = rider[0].boxes
    for box in boxes:
        cls = box.cls.item()
        cls = int(cls)
        conf = box.conf.item()
        conf = round(conf * 100, 2)
        class_name = class_labels[cls]
        label = f'{class_name}: {conf}%'

        x1,y1,x2,y2 = box.xyxy[0]
        x1,y1,x2,y2 = int(x1), int(y1), int(x2), int(y2)
        cv2.rectangle(original_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(original_img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        if class_name == 'Rider':
            X1,Y1,X2,Y2 = box.xyxy[0]
            X1,Y1,X2,Y2 = int(X1), int(Y1), int(X2), int(Y2)

        if class_name == 'Helmet':
            hex1,hey1,hex2,hey2 = box.xyxy[0]
            hex1,hey1,hex2,hey2 = int(hex1), int(hey1), int(hex2), int(hey2)
            if hex1 > X1 and hey1 > Y1 and hex2 < X2 and hey2 < Y2:
                helmet = True

        if class_name == 'Head':
            heax1,heay1,heax2,heay2 = box.xyxy[0]
            heax1,heay1,heax2,heay2 = int(heax1), int(heay1), int(heax2), int(heay2)
            if heax1 > X1 and heay1 > Y1 and heax2 < X2 and heay2 < Y2:
                head = True

        if class_name == 'Plate':
            px1,py1,px2,py2 = box.xyxy[0]
            px1,py1,px2,py2 = int(px1), int(py1), int(px2), int(py2)
            if px1 > X1 and py1 > Y1 and px2 < X2 and py2 < Y2:
                plate = True
                plate_img = original_img[py1:py2, px1:px2]
                text = reader.readtext(plate_img)
                for detection in text:
                    print("License Num:", detection[1].upper())
                    plate_text = detection[1].upper()
                    log_data(plate, helmet, head, plate_text)
        
        #print('head', head, 'helmet', helmet, 'plate', plate)

    head = False
    helmet = False
    plate = False

    cv2.imshow("Camera", original_img)
    #cv2.imshow("Plate", plate_img)
    if cv2.waitKey(50) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 41.4ms
Speed: 16.6ms preprocess, 41.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 35.5ms
Speed: 0.6ms preprocess, 35.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 32.7ms
Speed: 0.0ms preprocess, 32.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 30.9ms
Speed: 8.5ms preprocess, 30.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 30.5ms
Speed: 5.5ms preprocess, 30.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 27.9ms
Speed: 8.0ms preprocess, 27.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 34.2ms
Speed: 0.0ms preprocess, 34.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 30.0ms
Speed: 7.8ms preprocess, 30.0ms 